In [9]:
import requests as req
import pandas as pd

# options for viewing dataframes etc in jupyter
pd.set_option('max_rows', 300)
pd.set_option('max_columns', 20)
pd.set_option('max_colwidth', 200)

# ProPublica api get request for bills by subject 
get_bill_by_subj = 'https://api.propublica.org/congress/v1/bills/subjects/'
key = open("propubk.txt").read()
headers = { 'X-API-Key' : key, 'content-type':'application/json' }

def gatherAndPrint( subj, start, stop ):
    if type(subj) is not str:
        print("error: define subject(str), offset start (int), offset stop (int)")
    else:
        url = get_bill_by_subj + subj + '.json'
        print('getting '+url)
        
        # array to hold dataframes
        frames = []
        
        # if valid range of offsets ('stop' inclusive)
        if stop > start:
            # get each set of results, append to frame array, concat as 1 dataframe
            for o in range(start, stop +1):
                response = req.get(url, headers=headers, params={'offset': o*20 })
                responseDF = pd.read_json(response.text)
                results = pd.DataFrame.from_records(responseDF['results'])
                frames.append(results)
            billsDF = pd.concat(frames)
            filename = subj+'_bills_0'+str(start)+'-0'+str(stop)+'_offset'+str(start*20)+'-'+str(stop*20)+'.csv'
            billsDF.to_csv(filename)
            print('Combined and saved '+ str(len(frames))+ ' responses as '+filename) 
            billsDF
        
        # if a single offset (e.g. start 0, stop 0)
        elif stop - start == 0:
            response = req.get(url, headers=headers, params={'offset': start*20 })
            responseDF = pd.read_json(response.text)
            results = pd.DataFrame.from_records(responseDF['results'])
            filename = subj+'_bills_0'+str(start)+'_offset'+str(start*20)+'.csv'
            results.to_csv(filename)
            print('Saved offset '+ str(start*20) +' as '+filename) 
            results
        else:
            print('error: could not resolve offset')

# subject query examples: 'climate-change', 'coal'
gatherAndPrint('climate-change', 0, 4)

getting https://api.propublica.org/congress/v1/bills/subjects/global-warming.json
Combined and saved 10 responses as global-warming_bills_00-09_offset0-180.csv
